#### 层和块
回顾多层感知机

In [1]:
import torch
from torch import nn
# 定义了一些没有参数的函数
from torch.nn import functional as F

# 这里构造一个线性层+激活函数+线性层
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

# X是一个随机生成矩阵 2 批量大小 20输入维度
X = torch.rand(2,20)
net(X)

tensor([[-0.0290, -0.0548,  0.0060,  0.0044,  0.0147,  0.0257,  0.1951, -0.0426,
          0.0341, -0.0172],
        [ 0.1010, -0.0769,  0.0766,  0.0194, -0.1755,  0.0256,  0.1011,  0.1187,
          0.1113,  0.0254]], grad_fn=<AddmmBackward>)

#### nn.Sequential定义一种特殊的Module
Module可以被认为是任何一个层和一神经网络的父类

In [2]:
# 自定义一个与MLP一样的函数
class MLP(nn.Module): # 这里是继承Module类，得到一些好用的函数
    def __init__(self): # 函数一：定义需要的类和参数
        super().__init__() # 调用父类，将需要的内部参数设置好
        self.hidden = nn.Linear(20,256) # 将一层存储在类的成员变量中
        self.out = nn.Linear(256,10)  # 同理定义输出层
    
    def forward(self,X):    # 前置函数 定义前向计算的做法
        return self.out(F.relu(self.hidden(X))) # 定义输出的过程

#### 实例化多层感知机的各个层，然后每次调用正向传播函数时调用这些层

In [3]:
net = MLP() # 实例化这个类
net(X)      # 用X构造，得到输出2x10的矩阵

tensor([[ 0.0941,  0.1180, -0.0101, -0.3343, -0.1031,  0.1018, -0.0574, -0.1098,
          0.2197, -0.1258],
        [-0.0589,  0.2010,  0.0047, -0.1954, -0.0508, -0.0389, -0.0332,  0.0156,
          0.2268,  0.1484]], grad_fn=<AddmmBackward>)

#### 实现顺序块的类
与上面类似

In [8]:
class MySequential(nn.Module):
    def __init__(self,*args): # 接受一个list of arg，每个子类
        super().__init__()
        for block in args:    # 遍历每一个层
            self._modules[block] = block # pytorch定义了_modules容器，放的都是层
    
    def forward(self,X):
        for block in self._modules.values(): # 是有序的，直接就遍历赋值极客
            X = block(X)
        return X
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1573, -0.1332,  0.1710, -0.0105,  0.1711,  0.2546,  0.2754,  0.1214,
          0.0046, -0.0474],
        [ 0.2970, -0.0584,  0.1688, -0.0290,  0.2502,  0.1823,  0.3054,  0.0456,
          0.0027, -0.1731]], grad_fn=<AddmmBackward>)

#### 在正向传播函数中执行代码
继承，可以做成更多灵活操作

In [10]:
# 修改函数，增加需要的计算
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 增加一个不参与计算的权重层
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self,X):                  # 将X放进来
        X = self.linear(X)                # 先过一个线性层
        X = F.relu(torch.mm(X,self.rand_weight) +1 )   # 做一个矩阵乘法，然后偏执
        X = self.linear(X)
        while(X.abs().sum() > 1):   # 当绝对值求和大于1的时候，就循环除2
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1460, grad_fn=<SumBackward0>)

#### 混合搭配各种组合块的方法

In [14]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.1030, grad_fn=<SumBackward0>)